In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import logging 
from pathlib import Path
import time
from dash import Dash, Input, Output, State, callback, html, dcc
import plotly.graph_objs as go
import plotly.express as px


class DataHandler:
    
    def __init__(self, stocks=[], freq='quarterly'):
        
        if type(stocks) != type([]):
            stocks = [stocks]
        self.stocks = stocks
        
        self.logger = logging.getLogger(__name__)
        
        self.folder = Path('cache')
        if not self.folder.exists() or not self.folder.is_dir():
            self.folder.mkdir()
            
        self.freq = freq if freq in ['yearly', 'quarterly'] else 'quarterly'
        
        
    def fetch_data(self):
        df = pd.DataFrame()
        
        for stock in self.stocks:
            path = Path(f'{self.folder}/{stock}_fundamentals_{self.freq}.csv')
            
            if not path.exists():
                try:
                    ticker = yf.Ticker(stock)

                    cf_statement = ticker.get_cashflow(freq=self.freq)
                    balance_sheet = ticker.get_balancesheet(freq=self.freq)
                    income_statement = ticker.get_incomestmt(freq=self.freq)
                    df = pd.concat([cf_statement, balance_sheet, income_statement], join='inner', axis=0)
                    
                    if self.freq == 'quarterly':
                        df.columns = df.columns.to_period('Q').astype('str')
                    else:
                        df.columns = df.columns.year.astype('str')
                        
                    df.to_csv(path)

                except Exception as e:
                    self.logger.warning(e)
            else:
                df = pd.read_csv(path, index_col=0, parse_dates=True)
                
        return df
    
    def get_company_info(self, ticker):
        try:
            t = yf.Ticker(ticker)
            info = t.info
            return {
                'name': info.get('longName', ticker),
                'sector': info.get('sector', 'N/A'),
                'industry': info.get('industry', 'N/A'),
                'market_cap': info.get('marketCap', 0),
                'price': info.get('currentPrice', 0)
            }
        except:
            return {'name': ticker, 'sector': 'N/A', 'industry': 'N/A', 'market_cap': 0, 'price': 0}
    

class DataAnalyzer:
    def __init__(self, df=None):
        self.df = df if df is not None else pd.DataFrame()
        self.logger = logging.getLogger(__name__)
        
    def get_growth(self):
        fcf_margin = self.df.loc['FreeCashFlow'] / self.df.loc['TotalRevenue'] * 100
        net_margin = self.df.loc['NetIncome'] / self.df.loc['TotalRevenue'] * 100
        
        net_income = self.df.loc['NetIncome']
        total_revenue = self.df.loc['TotalRevenue']
        
        return {
            'net_m': net_margin,
            'fcf_m': fcf_margin,
            'tot_rev': total_revenue,
            'net_inc': net_income
        }
        
    def get_health(self):
        try:
            cash = self.df.loc['CashAndCashEquivalents']
            current_liabilities = self.df.loc['CurrentLiabilities']
            quick_ratio = cash / current_liabilities
        except KeyError:
            quick_ratio = pd.Series([0] * len(self.df.columns), index=self.df.columns)
        
        try:
            total_liabilities = self.df.loc['TotalLiabilitiesNetMinorityInterest']
        except KeyError:
            try:
                total_liabilities = self.df.loc['TotalDebt']
            except KeyError:
                total_liabilities = pd.Series([0] * len(self.df.columns), index=self.df.columns)
        
        try:
            total_assets = self.df.loc['TotalAssets']
        except KeyError:
            total_assets = pd.Series([0] * len(self.df.columns), index=self.df.columns)
        
        debt_to_assets = (total_liabilities / total_assets * 100) if not total_assets.eq(0).all() else pd.Series([0] * len(self.df.columns), index=self.df.columns)
        
        return {
            'qr': quick_ratio,
            'lia': total_liabilities,
            'ass': total_assets,
            'dta': debt_to_assets
        }
    
    def get_summary_metrics(self):
        """Berechnet wichtige Kennzahlen für die Summary Cards"""
        try:
            latest_revenue = self.df.loc['TotalRevenue'].dropna().iloc[-1]
            prev_revenue = self.df.loc['TotalRevenue'].dropna().iloc[-2] if len(self.df.loc['TotalRevenue'].dropna()) > 1 else latest_revenue
            revenue_growth = ((latest_revenue - prev_revenue) / prev_revenue * 100) if prev_revenue != 0 else 0
            
            latest_margin = (self.df.loc['NetIncome'] / self.df.loc['TotalRevenue'] * 100).dropna().iloc[-1]
            
            latest_fcf = self.df.loc['FreeCashFlow'].dropna().iloc[-1]
            prev_fcf = self.df.loc['FreeCashFlow'].dropna().iloc[-2] if len(self.df.loc['FreeCashFlow'].dropna()) > 1 else latest_fcf
            fcf_growth = ((latest_fcf - prev_fcf) / abs(prev_fcf) * 100) if prev_fcf != 0 else 0
            
            return {
                'revenue': latest_revenue / 1e9,
                'revenue_growth': revenue_growth,
                'margin': latest_margin,
                'fcf': latest_fcf / 1e9,
                'fcf_growth': fcf_growth
            }
        except:
            return {'revenue': 0, 'revenue_growth': 0, 'margin': 0, 'fcf': 0, 'fcf_growth': 0}


app = Dash(__name__)

# Farbschema
COLORS = {
    'background': '#0a0e27',
    'card_bg': '#151b3d',
    'card_border': '#1e2749',
    'text_primary': '#e8eaf6',
    'text_secondary': '#9ca3af',
    'accent_blue': '#3b82f6',
    'accent_green': '#10b981',
    'accent_red': '#ef4444',
    'accent_purple': '#8b5cf6',
    'accent_cyan': '#06b6d4',
    'gradient_start': '#667eea',
    'gradient_end': '#764ba2'
}

app.layout = html.Div([
    # Header Section
    html.Div([
        html.Div([
            html.H1(
                '📊 Fundamental Insights Pro',
                style={
                    'background': f'linear-gradient(135deg, {COLORS["gradient_start"]}, {COLORS["gradient_end"]})',
                    'WebkitBackgroundClip': 'text',
                    'WebkitTextFillColor': 'transparent',
                    'backgroundClip': 'text',
                    'fontWeight': '800',
                    'fontSize': '42px',
                    'margin': '0',
                    'letterSpacing': '-0.5px'
                }
            ),
            html.P(
                'Erweiterte Finanzanalyse und Visualisierung',
                style={
                    'color': COLORS['text_secondary'],
                    'fontSize': '16px',
                    'margin': '8px 0 0 0'
                }
            )
        ], style={'textAlign': 'center'}),
        
        # Search Bar
        html.Div([
            html.Div([
                dcc.Input(
                    type='text',
                    value='MSFT',
                    id='ticker',
                    placeholder='Ticker eingeben (z.B. AAPL, MSFT)',
                    style={
                        'width': '320px',
                        'padding': '16px 24px',
                        'fontSize': '16px',
                        'border': f'2px solid {COLORS["card_border"]}',
                        'borderRadius': '12px 0 0 12px',
                        'backgroundColor': COLORS['card_bg'],
                        'color': COLORS['text_primary'],
                        'outline': 'none',
                        'transition': 'all 0.3s ease'
                    }
                ),
                html.Button(
                    '🚀 Analysieren',
                    id='submit',
                    style={
                        'background': f'linear-gradient(135deg, {COLORS["accent_blue"]}, {COLORS["accent_purple"]})',
                        'color': '#ffffff',
                        'padding': '16px 32px',
                        'fontSize': '16px',
                        'fontWeight': '600',
                        'border': 'none',
                        'borderRadius': '0 12px 12px 0',
                        'cursor': 'pointer',
                        'transition': 'all 0.3s ease',
                        'boxShadow': '0 4px 12px rgba(59, 130, 246, 0.3)'
                    },
                    n_clicks=0
                )
            ], style={
                'display': 'flex',
                'justifyContent': 'center',
                'marginTop': '32px'
            })
        ])
    ], style={
        'padding': '48px 24px 32px 24px',
        'background': f'linear-gradient(180deg, {COLORS["background"]} 0%, {COLORS["card_bg"]} 100%)',
        'borderBottom': f'1px solid {COLORS["card_border"]}'
    }),
    
    # Company Info Cards
    html.Div(id='company-info', style={'padding': '24px'}),
    
    # Dashboard Selection
    html.Div([
        dcc.Dropdown(
            id='dashboard',
            options=[
                {'label': '📈 Wachstum & Profitabilität', 'value': 'growth'},
                {'label': '🫀 Finanzielle Gesundheit', 'value': 'health'}
            ],
            value='growth',
            style={
                'width': '400px',
                'fontSize': '15px',
                'fontWeight': '500',
                'backgroundColor': COLORS['card_bg'],
                'color': COLORS['text_primary'],
                'border': f'2px solid {COLORS["card_border"]}',
                'borderRadius': '12px'
            },
            clearable=False
        )
    ], style={
        'display': 'flex',
        'justifyContent': 'center',
        'padding': '0 24px 24px 24px'
    }),
    
    # Main Chart
    html.Div([
        dcc.Graph(
            id='graph',
            config={
                'displayModeBar': True,
                'displaylogo': False,
                'modeBarButtonsToRemove': ['pan2d', 'lasso2d', 'select2d']
            }
        )
    ], style={'padding': '0 24px 24px 24px'}),
    
], style={
    'backgroundColor': COLORS['background'],
    'minHeight': '100vh',
    'fontFamily': '-apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif'
})


@app.callback(
    Output('company-info', 'children'),
    [Input('submit', 'n_clicks')],
    State('ticker', 'value')
)
def update_company_info(n_clicks, ticker):
    if n_clicks == 0:
        return html.Div()
    
    dh = DataHandler(ticker)
    info = dh.get_company_info(ticker)
    data = dh.fetch_data()
    da = DataAnalyzer(data)
    metrics = da.get_summary_metrics()
    
    def create_metric_card(title, value, change, icon, color):
        return html.Div([
            html.Div([
                html.Span(icon, style={'fontSize': '28px', 'marginBottom': '8px'}),
                html.P(title, style={
                    'color': COLORS['text_secondary'],
                    'fontSize': '13px',
                    'fontWeight': '500',
                    'margin': '0',
                    'textTransform': 'uppercase',
                    'letterSpacing': '0.5px'
                }),
                html.H3(value, style={
                    'color': COLORS['text_primary'],
                    'fontSize': '28px',
                    'fontWeight': '700',
                    'margin': '8px 0',
                    'lineHeight': '1'
                }),
                html.Div([
                    html.Span('▲' if change >= 0 else '▼', style={
                        'color': COLORS['accent_green'] if change >= 0 else COLORS['accent_red'],
                        'fontSize': '12px',
                        'marginRight': '4px'
                    }),
                    html.Span(f'{abs(change):.1f}%', style={
                        'color': COLORS['accent_green'] if change >= 0 else COLORS['accent_red'],
                        'fontSize': '14px',
                        'fontWeight': '600'
                    })
                ])
            ])
        ], style={
            'backgroundColor': COLORS['card_bg'],
            'border': f'1px solid {COLORS["card_border"]}',
            'borderRadius': '16px',
            'padding': '24px',
            'flex': '1',
            'minWidth': '200px',
            'transition': 'all 0.3s ease',
            'borderLeft': f'4px solid {color}',
            'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)'
        })
    
    return html.Div([
        # Company Header
        html.Div([
            html.H2(info['name'], style={
                'color': COLORS['text_primary'],
                'fontSize': '24px',
                'fontWeight': '700',
                'margin': '0 0 8px 0'
            }),
            html.P(f"{info['sector']} • {info['industry']}", style={
                'color': COLORS['text_secondary'],
                'fontSize': '14px',
                'margin': '0'
            })
        ], style={'marginBottom': '24px'}),
        
        # Metrics Cards
        html.Div([
            create_metric_card(
                'Revenue',
                f'${metrics["revenue"]:.2f}B',
                metrics['revenue_growth'],
                '💰',
                COLORS['accent_blue']
            ),
            create_metric_card(
                'Net Margin',
                f'{metrics["margin"]:.1f}%',
                0,
                '📊',
                COLORS['accent_purple']
            ),
            create_metric_card(
                'Free Cash Flow',
                f'${metrics["fcf"]:.2f}B',
                metrics['fcf_growth'],
                '💵',
                COLORS['accent_green']
            ),
        ], style={
            'display': 'flex',
            'gap': '20px',
            'flexWrap': 'wrap'
        })
    ])


@app.callback(
    Output('graph', 'figure'),
    [Input('dashboard', 'value'), Input('submit', 'n_clicks')],
    State('ticker', 'value')
)
def update_graph(figure, n_clicks, ticker):
    
    dh = DataHandler(ticker)
    data = dh.fetch_data()
    da = DataAnalyzer(data)
    
    # Base Layout Template
    layout_template = {
        'template': 'plotly_dark',
        'paper_bgcolor': COLORS['card_bg'],
        'plot_bgcolor': COLORS['card_bg'],
        'font': dict(
            family='-apple-system, BlinkMacSystemFont, "Segoe UI", Roboto',
            size=13,
            color=COLORS['text_primary']
        ),
        'hovermode': 'x unified',
        'hoverlabel': dict(
            bgcolor=COLORS['background'],
            font_size=13,
            font_family='-apple-system, BlinkMacSystemFont, "Segoe UI", Roboto'
        ),
        'height': 600,
        'margin': dict(l=60, r=60, t=80, b=60),
        'xaxis': dict(
            showgrid=True,
            gridwidth=1,
            gridcolor=COLORS['card_border'],
            showline=True,
            linewidth=2,
            linecolor=COLORS['card_border']
        ),
        'yaxis': dict(
            showgrid=True,
            gridwidth=1,
            gridcolor=COLORS['card_border'],
            showline=True,
            linewidth=2,
            linecolor=COLORS['card_border']
        ),
        'legend': dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1,
            bgcolor='rgba(0,0,0,0)',
            font=dict(size=12)
        )
    }
    
    fig = go.Figure()
    
    if figure == 'growth':
        dic = da.get_growth()
        
        colors_bars = [COLORS['accent_blue'], COLORS['accent_cyan']]
        colors_lines = [COLORS['accent_green'], COLORS['accent_purple']]
        
        labels = {
            'tot_rev': 'Revenue',
            'net_inc': 'Net Income',
            'net_m': 'Net Margin',
            'fcf_m': 'FCF Margin'
        }
        
        bar_idx = 0
        line_idx = 0
        
        for key, value in dic.items():
            value = value.dropna()[::-1]
            if key in ['tot_rev', 'net_inc']:
                fig.add_trace(
                    go.Bar(
                        x=value.index,
                        y=value,
                        name=labels[key],
                        marker=dict(
                            color=colors_bars[bar_idx],
                            line=dict(width=0)
                        ),
                        hovertemplate='<b>%{x}</b><br>$%{y:.2f}B<extra></extra>',
                        opacity=0.9
                    )
                )
                bar_idx += 1
            
            if key in ['net_m', 'fcf_m']:
                fig.add_trace(
                    go.Scatter(
                        x=value.index,
                        y=value,
                        mode='lines+markers',
                        yaxis='y2',
                        name=labels[key],
                        line=dict(
                            width=3,
                            color=colors_lines[line_idx],
                            shape='spline'
                        ),
                        marker=dict(
                            size=10,
                            symbol='circle',
                            color=colors_lines[line_idx],
                            line=dict(width=2, color=COLORS['background'])
                        ),
                        hovertemplate='<b>%{x}</b><br>%{y:.2f}%<extra></extra>'
                    )
                )
                line_idx += 1
        
        layout_template.update({
            'title': dict(
                text=f'<b>{ticker}</b> - Wachstum & Profitabilität',
                font=dict(size=22, color=COLORS['text_primary']),
                x=0.5,
                xanchor='center'
            ),
            'yaxis': dict(
                title='Betrag (Milliarden $)',
                showgrid=True,
                gridwidth=1,
                gridcolor=COLORS['card_border']
            ),
            'yaxis2': dict(
                title='Marge (%)',
                overlaying='y',
                side='right',
                showgrid=False
            ),
            'barmode': 'group'
        })
        
    elif figure == 'health':
        dic = da.get_health()
        
        colors_bars = [COLORS['accent_red'], COLORS['accent_green']]
        colors_lines = [COLORS['accent_purple'], COLORS['accent_cyan']]
        
        labels = {
            'lia': 'Liabilities',
            'ass': 'Assets',
            'qr': 'Quick Ratio',
            'dta': 'Debt/Assets'
        }
        
        bar_idx = 0
        line_idx = 0
        
        for key, value in dic.items():
            value = value.dropna()[::-1]
            if key in ['lia', 'ass']:
                fig.add_trace(
                    go.Bar(
                        x=value.index,
                        y=value,
                        name=labels[key],
                        marker=dict(
                            color=colors_bars[bar_idx],
                            line=dict(width=0)
                        ),
                        hovertemplate='<b>%{x}</b><br>$%{y:.2f}B<extra></extra>',
                        opacity=0.9
                    )
                )
                bar_idx += 1
            
            if key in ['qr', 'dta']:
                fig.add_trace(
                    go.Scatter(
                        x=value.index,
                        y=value,
                        mode='lines+markers',
                        yaxis='y2',
                        name=labels[key],
                        line=dict(
                            width=3,
                            color=colors_lines[line_idx],
                            shape='spline'
                        ),
                        marker=dict(
                            size=10,
                            symbol='circle',
                            color=colors_lines[line_idx],
                            line=dict(width=2, color=COLORS['background'])
                        ),
                        hovertemplate='<b>%{x}</b><br>%{y:.2f}<extra></extra>' if key == 'qr' else '<b>%{x}</b><br>%{y:.2f}%<extra></extra>'
                    )
                )
                line_idx += 1
        
        layout_template.update({
            'title': dict(
                text=f'<b>{ticker}</b> - Finanzielle Gesundheit',
                font=dict(size=22, color=COLORS['text_primary']),
                x=0.5,
                xanchor='center'
            ),
            'yaxis': dict(
                title='Betrag (Milliarden $)',
                showgrid=True,
                gridwidth=1,
                gridcolor=COLORS['card_border']
            ),
            'yaxis2': dict(
                title='Ratio / %',
                overlaying='y',
                side='right',
                showgrid=False
            ),
            'barmode': 'group'
        })
    
    fig.update_layout(layout_template)
    
    return fig


if __name__ == '__main__':
    logging.basicConfig(level=logging.WARNING)
    app.run(debug=True)